<a href="https://colab.research.google.com/github/bishair/Model/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from google.colab import files
import os

In [18]:
#Function to determine if the user uploaded csv or xlsx file format
def load_data(file_name):
    # Determine the file extension
    file_extension = os.path.splitext(file_name)[1]
    # Load data based on file extension
    if file_extension == '.xlsx':
        return pd.read_excel(file_name)
    elif file_extension == '.csv':
        return pd.read_csv(file_name)
    else:
        raise ValueError("Unsupported file type. Please upload an Excel or CSV file.")

#Function that returns the index of the row where the header is located based on the provided keywords
def find_header_row(file_name, keywords):
    # Load only the first 10 rows of the file for efficiency
    data = load_data(file_name).head(10)
    # Iterate over the rows to find the header row
    for row_index, row in data.iterrows():
        # Iterate over each cell in the row. For each cell iterate over each keyword in keywords. Covert each cell to string and to lowercase for efficiency
        if any(keyword.lower() in str(cell).lower() for cell in row for keyword in keywords):
            # Return the index of the header row
            return row_index
    # If no header row is found, return None
    return None

#Function to determine which columns contain the data on GW level, date and river water level
def find_columns_by_keyword(dataframe, keywords):
    matched_columns = []
    for column in dataframe.columns:
        # Ensure the column name is a string before calling .lower()
        column_str = str(column)
        if any(keyword.lower() in column_str.lower() for keyword in keywords):
            matched_columns.append(column)
    return matched_columns


In [9]:
# Upload the original file
uploaded = files.upload()
#In case user upload more than one file, only take the name of the first file
file_name = next(iter(uploaded))

Saving data of 2014.xlsx to data of 2014 (1).xlsx


In [10]:
# Keywords to look for in the header row  =>  MODIFYY LATER DEPENDING UPON SAMPLE
header_keywords = ["date", "max", "min", "G21"]
data_keywords = ["date", "river", "water", "level", "groundwater", "G"]

In [16]:
# Find the header row
header_row = find_header_row(file_name, header_keywords)

# Check if the header row was found
if header_row is None:
    raise ValueError("Could not identify the header row. Please check the file format.")

# Load the data with the correct header row
data = load_data(file_name)
if header_row is not None:
    data.columns = data.iloc[header_row]
    data = data.drop(data.index[0:header_row+1]).reset_index(drop=True)  #Remove all the rows above the header (header is removed as well). Reset the index to 0

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [19]:
# Identify columns based on the keywords
date_columns = find_columns_by_keyword(data, ["date", "time"])
river_level_columns = find_columns_by_keyword(data, ["river", "water", "level", "max", "min"])
groundwater_level_columns = find_columns_by_keyword(data, ["groundwater", "G"])

In [21]:
# Check if we found exactly one column for each
if len(date_columns) != 1 or len(river_level_columns) < 2 or len(groundwater_level_columns) == 0:
    raise ValueError("Could not uniquely identify required columns. Please check the file format.")

# Extract the relevant columns
date_column = date_columns[0]
river_water_level_max = data[river_level_columns[0]].astype(float)  # Assuming first max
river_water_level_min = data[river_level_columns[1]].astype(float)  # Assuming second min
groundwater_level = data[groundwater_level_columns[0]].astype(float)  # Assuming first groundwater column

# Calculate the average river water level
river_water_level_avg = (river_water_level_max + river_water_level_min) / 2

# Handle missing values (if any) by forward filling
river_water_level_avg.fillna(method='ffill', inplace=True)
groundwater_level.fillna(method='ffill', inplace=True)

# Create a DataFrame with the preprocessed data
preprocessed_data = pd.DataFrame({
    'Date': pd.to_datetime(data[date_column]),
    'River_Water_Level_Avg': river_water_level_avg,
    'Groundwater_Level': groundwater_level
})

# Export the preprocessed data to a new Excel file
preprocessed_file_name = 'preprocessed_data_with_dates.xlsx'
preprocessed_data.to_excel(preprocessed_file_name, index=False)

# Download the preprocessed data file
files.download(preprocessed_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>